# **Tracking degradation of local Fermi liquid**

In [ ]:
import itertools
from tqdm import tqdm
from time import sleep
from math import sqrt
from multiprocessing import Pool
from itertools import product, repeat
import matplotlib as mpl
from matplotlib import pyplot as plt
import numpy as np
from qutip import *
from scipy.signal import peak_widths
import scipy.optimize
from functools import partial
import warnings
from scipy.sparse.linalg import eigsh
from scipy.linalg import eigh
from numpy.linalg import eigvalsh
import os, sys
from os import devnull

font = {'size'   : 30}

mpl.rc('font', **font)
mpl.rcParams['text.usetex'] = True
plt.rc('text.latex', preamble=r'\usepackage{amsmath,nicefrac}')
plt.rcParams["figure.figsize"]= 10, 8
mpl.rcParams['lines.linewidth'] = 2
props = dict(boxstyle='round', ec='black', fc='white', alpha=0.5)

deltaD = 0.5
rho = 2/deltaD
nj = rho*deltaD

corr_folder = './corr_data/'
fig_folder = './figs/'
lfl_wf_folder = './wfs_for_entanglement/'

for folder in (corr_folder, fig_folder, lfl_wf_folder):
    if not os.path.exists(folder): os.mkdir(folder)

In [ ]:
def get_full_ham(t, lattice_dim, U, V, J):
    imp_length = 1
    total_dim = 2*(imp_length + lattice_dim)
    c_all = [tensor([sigmaz()]*(i) + [destroy(2)] + [identity(2)]*(total_dim - i -1)) for i in range(total_dim)]
    
    Sz_imp = 0.5 * (c_all[0].dag()*c_all[0] - c_all[1].dag()*c_all[1])
    Sp_imp = c_all[0].dag()*c_all[1]
    Sm_imp = Sp_imp.dag()
    Sz_bath = 0.5 * (c_all[2].dag()*c_all[2] - c_all[3].dag()*c_all[3])
    Sp_bath = c_all[2].dag()*c_all[3]
    Sm_bath = Sp_bath.dag()
    
    H_U = (-U*2) * Sz_imp**2
    H_V = V * (c_all[0].dag() * c_all[2] + c_all[1].dag() * c_all[3])
    H_J = J * (Sz_imp * Sz_bath + 0.5 * Sp_imp * Sm_bath + 0.5 * Sm_imp * Sp_bath)
    H_t = 0
    for i in range(2*imp_length, total_dim-2):
#         if i >= total_dim-2 and lattice_dim <= 2: continue
#         j = i+2-2*lattice_dim if i >= total_dim-2 and lattice_dim > 2 else i + 2
        j = i + 2
        H_t += -t * c_all[i].dag() * c_all[j]

    return H_U + H_J + H_V + H_V.dag() + H_t + (H_t.dag() if H_t != 0 else 0)

In [ ]:
def get_ham_K(Ek_0, ed, U, V, J):
    '''Create Hamiltonian in k-space'''
 
    assert ed == -U/2
    dim = len(Ek_0)
    total_dim = 2 * (dim + 1)
    c_all = [tensor([sigmaz()]*(i) + [destroy(2)] + [identity(2)]*(total_dim - i -1)) for i in range(total_dim)]
    Sz_imp = 0.5 * (c_all[0].dag()*c_all[0] - c_all[1].dag()*c_all[1])
    Sp_imp = c_all[0].dag()*c_all[1]
    Sm_imp = Sp_imp.dag()
    
    H_U = (-U*2) * Sz_imp**2
    H_J = 0
    H_K = 0
    H_V = 0
    for i in range(1, dim+1):
        H_K += Ek_0[i-1]*(c_all[2*i].dag()*c_all[2*i] + c_all[2*i+1].dag()*c_all[2*i+1])
        H_V += V * (c_all[0].dag()*c_all[2*i] + c_all[1].dag()*c_all[2*i+1])
        for j in range(1, dim+1):
            Sz_ij = 0.5 * (c_all[2*i].dag()*c_all[2*j] - c_all[2*i+1].dag()*c_all[2*j+1])
            Sp_ij = c_all[2*i].dag()*c_all[2*j+1]
            Sm_ij = c_all[2*i+1].dag()*c_all[2*j]
            H_J += J * (Sz_imp*Sz_ij + 0.5*Sp_imp*Sm_ij + 0.5*Sm_imp*Sp_ij)

    return H_U + H_K + H_V + H_V.dag() + H_J

In [ ]:
def leh_coeffs(params):
    t, U, V, J = params
    Delta = sqrt(U**2 + 16*V**2)/2
    vgamma = (3*J/4 + U/2)/(2)
    cs_p, cs_m = sqrt(sqrt(vgamma**2 + 4*V**2) - vgamma)/sqrt(2*sqrt(vgamma**2 + 4*V**2)), sqrt(sqrt(vgamma**2 + 4*V**2) + vgamma)/sqrt(2*sqrt(vgamma**2 + 4*V**2))
    cc_p, cc_m = -cs_m, cs_p
    E_gs = -3*J/8 - U/4 - sqrt(vgamma**2 + 4*V**2)
    E_CS = 0
    E_00 = 0
    E_uu = E_ST = -U/2 + J/4
    E_2 = -3*J/8 - U/4 + sqrt(vgamma**2 + 4*V**2)
    E_pm = np.array([-U/4 + Delta/2, -U/4 - Delta/2])
    N_pm = np.array([sqrt(16*V**2 + (U + 2*Delta)**2), sqrt(16*V**2 + (U - 2*Delta)**2)])
    
    delta_pm = E_pm - E_gs
    delta_CS = E_CS - E_gs
    delta_00 = E_00 - E_gs
    delta_uu = E_uu - E_gs
    delta_2 = E_2 - E_gs
    delta_ST = E_ST - E_gs
    a1 = 4*V/(sqrt(2)*N_pm)
    a2 = np.array([U+2*Delta, U-2*Delta])/(sqrt(2)*N_pm)
    Sm = cs_m*a1 + cc_m*a2
    Sp = cs_p*a1 + cc_p*a2
    Dm = cs_m*a1 - cc_m*a2
    Dp = cs_p*a1 - cc_p*a2
    
    X1 = sum(Sm**2/delta_pm)
    X2 = sum(Sm**2/delta_pm**2)
    Y1 = sum(Dm**2/delta_pm)
    Y2 = sum(Dm**2/delta_pm**2)
    P = sum(Sp*Sm/delta_pm)
    Q = sum(Dp*Dm/delta_pm)
    F1 = sum(a1*Sm/delta_pm)
    F2 = sum(a2*Sm/delta_pm)
    G1 = sum(a1*Dm/delta_pm)
    G2 = sum(a2*Dm/delta_pm)
    
    coeff_charge = -4*t**4 * (t**(-2)*X1/2 + F2**2 * (1/delta_CS + 2/delta_00) + P**2 / delta_2 - X1 * X2)
    coeff_spin = -t**4 * (t**(-2)*(X1 + Y1) + (F1**2 + G1**2)*(1/delta_ST + 1/delta_uu) + (F2**2 + G2**2)/delta_CS + (P**2 + Q**2)/delta_2 - (X1 + Y1)*(X2 + Y2))
    max_frac = max((max(t**2/abs(delta_pm)), t**2/abs(delta_CS), t**2/abs(delta_00), t**2/abs(delta_uu), t**2/abs(delta_2), t**2/abs(delta_ST)))
    
    return coeff_charge - coeff_spin, max_frac, np.min((min(delta_pm), delta_CS, delta_00, delta_uu, delta_2, delta_ST))

## Real space measures of entanglement

In [ ]:
def get_MI(Xgs, i, j):
    rho = Xgs * Xgs.dag()
    rho_part = rho.ptrace(i+j)
    I = entropy_mutual(rho_part, (0,1), (2,3))
    return I


def get_EE(Xgs, i):
    rho = Xgs * Xgs.dag()
    rho_part = rho.ptrace(i)
    EE = entropy_vn(rho_part)
    return EE


def get_corr(Xgs, i, j):
    total_dim = int(len(Xgs.dims[0]))
    c_all = [tensor([sigmaz()]*(i) + [destroy(2)] + [identity(2)]*(total_dim - i -1)) for i in range(total_dim)]
    
    Sz = [0.5 * (c_all[2*k].dag()*c_all[2*k] - c_all[2*k+1].dag()*c_all[2*k+1]) for k in (i,j)]
    Sp = [c_all[2*k].dag()*c_all[2*k+1] for k in (i,j)]
    Sm = [Sp_k.dag() for Sp_k in Sp]
    
    zz = Sz[0] * Sz[1]
    pm = Sp[0] * Sm[1]
    mp = Sm[0] * Sp[1]
    
    op = Sz[0] * Sz[1] + 0.5 * Sp[0] * Sm[1] + 0.5 * Sm[0] * Sp[1]
    return np.array(Xgs.dag() * op * Xgs)[0][0]


def get_chargecorr(Xgs, i, j):
    dim = int(len(Xgs.dims[0])/2)
    Cz = 0.5*(tensor(create(2) * destroy(2), identity(2)) + tensor(identity(2), create(2) * destroy(2)) - 1)
    Cp = tensor(create(2), create(2))
    Cm = Cp.dag()
    zz = tensor([identity(2)]*2*i + [Cz] + [identity(2)]*2*(j-i-1) + [Cz] + [identity(2)]*2*(dim - j - 1))
    pm = tensor([identity(2)]*2*i + [Cp] + [identity(2)]*2*(j-i-1) + [Cm] + [identity(2)]*2*(dim - j - 1))
    mp = tensor([identity(2)]*2*i + [Cm] + [identity(2)]*2*(j-i-1) + [Cp] + [identity(2)]*2*(dim - j - 1))    
    op = zz + 0.5 * (pm + mp)
    return np.array(Xgs.dag() * op * Xgs)[0][0]


def get_csm(params):
    U,V,J = params
    vgamma = (3*J/4 + U/2)/(2)
    cs_m = sqrt(sqrt(vgamma**2 + 4*V**2) + vgamma)/sqrt(2*sqrt(vgamma**2 + 4*V**2))
    cc_m = sqrt(1 - cs_m**2)
    return cs_m, cc_m


def get_twosite_EEimp(params):
    csm, ccm = get_csm(params)
    return -(csm**2 * np.log(csm**2/2) + ccm**2 * np.log(ccm**2/2))


def get_tb_MI(params):
    U, V, J = params
    V = J = 0
    H = get_full_ham(t, lattice_dim, U, V, J)
    H = np.array(H)
    E, X = eigsh(H, k=1, which='SA')
    Xgs_r = Qobj(X[:, 0], dims=tensor([basis(2,0)]*2*(lattice_dim+1)).dims)
    return get_MI(Xgs_r, (2,3), (4,5))

### Correlations for Gen SIAM

In [ ]:
t = 0.1
lattice_dim = 4
U_ = 10**np.linspace(-2.2, 3, 50)
J_ = 1/np.copy(U_)
V_ = np.copy(J_)
I_01 = []
for U,J,V in tqdm(zip(U_,J_,V_), total=len(U_)):
    H = get_full_ham(t, lattice_dim, U, V, J)
    E, X = H.eigenstates()
#     if sum(E == min(E)) > 1:
#         print (U)
#         continue
#     H = np.array(H)
#     _, X = eigsh(H, k=1, which='SA')
#     Xgs_r = Qobj(X[:, 0], dims=tensor([basis(2,0)]*2*(lattice_dim+1)).dims)
    I_01.append(get_MI(X[0], (4,5), (0, 1)))
    plt.scatter(U, I_01[-1])
plt.xscale("log")

### Correlations for SIAM

In [ ]:
U_ = 10**np.linspace(-2.2, 2.5, 200)
J_ = 0*np.copy(U_)
V_ = 1/np.copy(U_)
text_str = create_text(('t', 'UV', 'J'), (t, 1, 0))
plot_all(t, U_, V_, J_, lattice_dim, U_/V_, r"$\nicefrac{U}{V}$", text_str, log_scale_y=True,
         save="t={:.3f},V=1_over_U,J=0,N={:.0f},U={:.3f},{:.3f},{:.0f}".format(t, lattice_dim, U_[0], U_[-1], len(U_)))

### Correlations for Kondo

In [ ]:
J_ = 10**np.linspace(-2.2, 2.5, 200)
U_ = 0*np.copy(J_)
V_ = 0*np.copy(J_)
text_str = create_text(('t', 'U', 'V'), (t, 0, 0))
plot_all(t, U_, V_, J_, lattice_dim, U_/J_, r"$\nicefrac{U}{J}$", text_str, log_scale_y=True,
         save="t={:.3f},U=0,V=0,N={:.0f},J={:.3f},{:.3f},{:.0f}".format(t, lattice_dim, J_[0], J_[-1], len(J_)))